# Corey Book - Lightweight Character Generation

**Memory-optimized version** that won't crash Colab's free tier.

Uses Stable Diffusion 1.5 + ControlNet (much smaller than SDXL) with your reference images.

**Features:**
- Uses your character reference images
- Lightweight models (fits in Colab free tier)
- Character consistency through ControlNet
- **Cost: FREE**

## 1. Lightweight Setup

In [ ]:
# Fix version conflicts by installing compatible versions
print("📦 Installing compatible packages...")

# First, upgrade huggingface_hub to fix the compatibility issue
!pip install -q --upgrade huggingface_hub

# Install compatible versions together
!pip install -q diffusers==0.25.0 transformers==4.36.0 accelerate==0.25.0
!pip install -q controlnet-aux opencv-python pillow

print("✅ Compatible installation complete!")
print("🔄 Please restart runtime and run this cell again to avoid conflicts")

In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from pathlib import Path

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from controlnet_aux import CannyDetector

# Check available memory
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name()}")
    print(f"🔋 GPU Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
    
    # Clear any existing GPU memory
    torch.cuda.empty_cache()
    
    device = "cuda"
else:
    print("⚠️ No GPU detected - using CPU (very slow)")
    device = "cpu"

## 2. Upload Files

Upload your reference images and page prompts:

In [ ]:
# Create directories
os.makedirs('cartoon-characters', exist_ok=True)
os.makedirs('page-prompts', exist_ok=True)
os.makedirs('generated_images', exist_ok=True)

print("📁 Upload these files using the file browser (left panel):")
print("   1. cartoon-characters/corey1.jpg")
print("   2. page-prompts/page-00-cover.md through page-03.md")
print("\n💡 Tip: You can drag and drop files directly!")

In [ ]:
# Check files
required = [
    'cartoon-characters/corey1.jpg',
    'page-prompts/page-00-cover.md'
]

all_good = True
for file in required:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - please upload")
        all_good = False

# Show all page files found
page_files = sorted([f for f in os.listdir('page-prompts/') if f.endswith('.md')])
print(f"\n📄 Found {len(page_files)} page files: {page_files}")

if all_good:
    print("\n🎉 Ready to generate!")
else:
    print("\n⚠️ Upload missing files before continuing")

## 3. Load Lightweight Models

Using SD 1.5 instead of SDXL to save memory:

In [ ]:
print("📦 Loading lightweight models...")

# Load ControlNet (Canny for edge detection)
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

# Load SD 1.5 pipeline (much smaller than SDXL)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,  # Save memory
    requires_safety_checker=False
)

# Memory optimizations
if device == "cuda":
    pipe = pipe.to("cuda")
    pipe.enable_attention_slicing()  # Reduce memory usage
    pipe.enable_model_cpu_offload()  # Move models to CPU when not needed

# Load Canny detector
canny_detector = CannyDetector()

print("✅ Lightweight models loaded!")
print(f"🧠 Memory usage reduced - should fit in Colab free tier")

## 4. Generation Functions

In [ ]:
def load_page_prompt(file_path):
    """Load page prompt from markdown file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    lines = content.split('\n')
    title = lines[0].replace('# ', '') if lines else "Unknown Page"
    
    # Extract image prompt
    image_prompt = ""
    in_image_prompt = False
    
    for line in lines:
        if line.startswith("## IMAGE PROMPT"):
            in_image_prompt = True
            continue
        elif line.startswith("## "):
            in_image_prompt = False
            continue
        
        if in_image_prompt and line.strip():
            image_prompt += line.strip() + " "
    
    return {
        'title': title.strip(),
        'image_prompt': image_prompt.strip()
    }

def create_prompt(page_data):
    """Create optimized prompt."""
    prompt = "children's book illustration, cartoon style, "
    prompt += "Corey: completely bald chef, no hair, round friendly face, navy blue apron, "
    prompt += page_data['image_prompt']
    prompt += " vibrant colors, cel shading, professional illustration, high quality"
    return prompt

def prepare_control_image(ref_path, target_size=(512, 512)):
    """Prepare reference image for ControlNet."""
    if not os.path.exists(ref_path):
        print(f"❌ Reference not found: {ref_path}")
        return None, None
        
    # Load and resize reference
    ref_image = Image.open(ref_path).convert('RGB')
    ref_image = ref_image.resize(target_size)
    
    # Generate Canny edges
    canny_image = canny_detector(ref_image)
    
    return ref_image, canny_image

print("🛠️ Helper functions loaded!")

In [ ]:
def generate_image(page_data, reference_path="cartoon-characters/corey1.jpg"):
    """Generate image with character consistency."""
    
    print(f"🎨 Generating: {page_data['title']}")
    
    # Prepare control image
    ref_image, canny_image = prepare_control_image(reference_path)
    if ref_image is None:
        return None
    
    # Create prompt
    prompt = create_prompt(page_data)
    negative_prompt = "low quality, blurry, deformed, extra limbs, bad anatomy, text, watermark"
    
    print(f"📝 Prompt: {prompt[:80]}...")
    
    # Clear memory before generation
    if device == "cuda":
        torch.cuda.empty_cache()
    
    try:
        # Generate with ControlNet
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=canny_image,
            controlnet_conditioning_scale=0.8,
            num_inference_steps=20,  # Fewer steps to save time/memory
            guidance_scale=7.5,
            width=512,  # Smaller size for memory
            height=512,
            generator=torch.manual_seed(42)  # Consistent seed
        )
        
        generated_image = result.images[0]
        
        # Upscale to 1024x1024 for final output
        generated_image = generated_image.resize((1024, 1024), Image.Resampling.LANCZOS)
        
        return generated_image, ref_image, canny_image
        
    except Exception as e:
        print(f"❌ Generation failed: {e}")
        # Clear memory on error
        if device == "cuda":
            torch.cuda.empty_cache()
        return None

print("🚀 Generation function ready!")

## 5. Test Generation

In [ ]:
# Test with cover page
if os.path.exists('page-prompts/page-00-cover.md'):
    print("🧪 Testing with cover page...")
    
    page_data = load_page_prompt('page-prompts/page-00-cover.md')
    result = generate_image(page_data)
    
    if result:
        generated, reference, canny = result
        
        # Show results
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        
        axes[0].imshow(reference)
        axes[0].set_title("Reference")
        axes[0].axis('off')
        
        axes[1].imshow(canny, cmap='gray')
        axes[1].set_title("Control (Canny)")
        axes[1].axis('off')
        
        axes[2].imshow(generated)
        axes[2].set_title("Generated")
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Save test result
        generated.save('generated_images/test-cover.png')
        print("💾 Test saved as: generated_images/test-cover.png")
        
    else:
        print("❌ Test failed")
else:
    print("❌ No cover page found - upload page-prompts/page-00-cover.md")

## 6. Generate All Available Pages

In [ ]:
# Find all page files
page_files = sorted([f for f in os.listdir('page-prompts/') if f.endswith('.md')])

if not page_files:
    print("❌ No page files found. Upload some .md files to page-prompts/")
else:
    print(f"🎨 Generating {len(page_files)} pages...")
    print(f"💰 Cost: FREE (Colab GPU)")
    
    results = []
    
    for i, page_file in enumerate(page_files, 1):
        print(f"\n🖼️ [{i}/{len(page_files)}] Processing {page_file}...")
        
        try:
            # Load page
            page_data = load_page_prompt(f'page-prompts/{page_file}')
            
            # Generate
            result = generate_image(page_data)
            
            if result:
                generated, _, _ = result
                
                # Save
                output_name = page_file.replace('.md', '.png')
                output_path = f'generated_images/{output_name}'
                generated.save(output_path)
                
                results.append(output_path)
                print(f"✅ Saved: {output_path}")
                
                # Show result
                plt.figure(figsize=(6, 6))
                plt.imshow(generated)
                plt.title(page_data['title'])
                plt.axis('off')
                plt.show()
                
            else:
                print(f"❌ Failed: {page_file}")
                
        except Exception as e:
            print(f"❌ Error with {page_file}: {e}")
            # Clear memory and continue
            if device == "cuda":
                torch.cuda.empty_cache()
    
    print(f"\n🎉 Complete! Generated {len(results)} images")
    print(f"📁 Check the generated_images/ folder")

## 7. Download Results

In [ ]:
import zipfile
from datetime import datetime

# Create zip with all images
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_name = f"corey_book_lightweight_{timestamp}.zip"

with zipfile.ZipFile(zip_name, 'w') as zipf:
    for file in os.listdir('generated_images/'):
        if file.endswith('.png'):
            zipf.write(f'generated_images/{file}', file)

print(f"📦 Created: {zip_name}")
print(f"💾 Right-click in file browser to download")

# Show what was generated
image_files = [f for f in os.listdir('generated_images/') if f.endswith('.png')]
print(f"\n📊 Generated {len(image_files)} images:")
for img in sorted(image_files):
    size_mb = os.path.getsize(f'generated_images/{img}') / (1024*1024)
    print(f"  {img}: {size_mb:.1f} MB")

## Memory-Saving Tips

**This lightweight version:**
- ✅ Uses SD 1.5 (smaller than SDXL)
- ✅ Generates at 512x512, upscales to 1024x1024
- ✅ Enables attention slicing & CPU offloading
- ✅ Clears GPU memory between generations
- ✅ Should work on Colab free tier

**Character Consistency:**
- Uses your `corey1.jpg` as structural reference
- ControlNet ensures similar poses/proportions
- Much better than text-only approaches

**If it still crashes:**
- Try generating 1-2 images at a time
- Restart runtime between batches
- Use Colab Pro for more RAM ($9.99/month)